In [20]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
!pip install wandb
import wandb
wandb.login(key="99ed1e6d8f514ee3823dec88049f21d48e678419")
import random
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [22]:
device

device(type='cuda')

In [23]:
import csv

def read_dataset(file_path):
    pairs = []
    with open(file_path, 'r', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            romanized_str = row[0].strip()
            devanagari_str = row[1].strip()
            romanized_str='['+romanized_str+']'
            devanagari_str='['+devanagari_str+']'
            pairs.append((romanized_str, devanagari_str))
    return pairs

In [24]:
file_path='/kaggle/input/aksrrrrrrrrrrrr/aksharantar_sampled/hin/hin_train.csv'
pairs=read_dataset(file_path)

In [25]:
file_path='/kaggle/input/aksrrrrrrrrrrrr/aksharantar_sampled/hin/hin_valid.csv'
valid_pairs=read_dataset(file_path)
file_path='/kaggle/input/aksrrrrrrrrrrrr/aksharantar_sampled/hin/hin_test.csv'
test_pairs=read_dataset(file_path)

In [26]:
type(pairs[0][0])

str

In [27]:
SOS_token = '['
EOS_token = ']'


In [28]:
native_words=[]
devgiri_words=[]

def prepare_vocabulary(pairs):

  native_character_vocabulary=set()
  latin_character_vocabulary=set()
  '''native_character_vocabulary.add('#')
  native_character_vocabulary.add('[')
  
  latin_character_vocabulary.add('#')
  latin_character_vocabulary.add('[')
  native_character_vocabulary.add(']')
  latin_character_vocabulary.add(']')'''
  

  native_enc_map={}
  latin_enc_map={}
  for (x,y) in pairs:
    native_character_vocabulary=native_character_vocabulary.union(set(list(x)))
    latin_character_vocabulary=latin_character_vocabulary.union(set(list(y)))
    

  #native_character_vocabulary=sorted(list(native_character_vocabulary))
  #latin_character_vocabulary=sorted(list(latin_character_vocabulary))

  

  i=0
  #native_char_map[SOS_token]=0
  #native_char_map[EOS_token]=0
  native_enc_map['#']=0
  native_enc_map['[']=1
  native_enc_map[']']=2
  i=3
  for ch in native_character_vocabulary:
    if ch not in {'#','[',']'}:
      native_enc_map[ch]=i
      i+=1
  
  native_enc_map['?']=i
  
  #latin_char_map[SOS_token]=0
  native_dec_map = {v: k for k, v in native_enc_map.items()}

  i=0
  latin_enc_map['#']=0
  latin_enc_map['[']=1
  latin_enc_map[']']=2
  i=3
  for ch in latin_character_vocabulary:
    if ch not in {'#','[',']'}:
      latin_enc_map[ch]=i
      i+=1
  latin_enc_map['?']=i
  native_character_vocabulary.add('#')
  native_character_vocabulary.add('[')
  
  latin_character_vocabulary.add('#')
  latin_character_vocabulary.add('[')
  native_character_vocabulary.add(']')
  latin_character_vocabulary.add(']')  
  #latin_char_map[EOS_token]=i
  latin_dec_map = {v: k for k, v in latin_enc_map.items()}

  return native_character_vocabulary,latin_character_vocabulary,native_enc_map,latin_enc_map,native_dec_map,latin_dec_map,pairs


native_character_vocabulary,latin_character_vocabulary,native_enc_map,latin_enc_map,native_dec_map,latin_dec_map,pairs=prepare_vocabulary(pairs)






In [29]:
native_enc_map

{'#': 0,
 '[': 1,
 ']': 2,
 'h': 3,
 's': 4,
 'u': 5,
 'a': 6,
 'l': 7,
 'p': 8,
 'v': 9,
 'c': 10,
 'f': 11,
 'o': 12,
 'b': 13,
 'z': 14,
 'w': 15,
 'i': 16,
 'd': 17,
 'k': 18,
 'q': 19,
 'n': 20,
 'm': 21,
 'r': 22,
 'g': 23,
 'x': 24,
 't': 25,
 'e': 26,
 'y': 27,
 'j': 28,
 '?': 29}

In [30]:
for x,y in pairs:
  print(len(x),x)
  break

13 [shastragaar]


In [31]:
max_encoder_seq_length = max([len(x) for x,y in pairs])
max_decoder_seq_length = max([len(y) for x,y in pairs])


In [32]:
print(max_encoder_seq_length)
print(max_decoder_seq_length)

26
22


In [33]:
import numpy as np
from torch.nn.utils.rnn import pad_sequence
def prepare_encoding(input,native_char_map,latin_char_map,batch_size):
  #input_encoding = np.zeros((M, max_encoder_seq_length, num_input_tokens + 1), dtype="float")
  input_encoding=[torch.tensor([native_char_map[ch] for ch in train_pair[0]],device=device) for train_pair in input]
  #input_encoding.append(EOS_token)
  input_encoding = pad_sequence(input_encoding, batch_first=True,padding_value=0)
  input_encoding=torch.tensor(input_encoding).to(device)
  target_encoding=[torch.tensor([latin_char_map[ch] for ch in train_pair[1]],device=device)for train_pair in input]
  #target_encoding.append(EOS_token)
  target_encoding = pad_sequence(target_encoding, batch_first=True,padding_value=0)
  target_encoding=torch.tensor(target_encoding).to(device)

  return input_encoding.T,target_encoding.T


input_encoding,target_encoding=prepare_encoding(pairs[0:10],native_enc_map,latin_enc_map,10)

/tmp/ipykernel_31/3310807116.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_encoding=torch.tensor(input_encoding).to(device)
/tmp/ipykernel_31/3310807116.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_encoding=torch.tensor(target_encoding).to(device)


In [34]:
from IPython.core.displayhook import Float
import numpy as np
from torch.nn.utils.rnn import pad_sequence
def prepare_encoding(input,native_char_map,latin_char_map,batch_size):
  #input_encoding = np.zeros((M, max_encoder_seq_length, num_input_tokens + 1), dtype="float")
  input_encoding=[torch.tensor([native_char_map[ch] if ch in native_character_vocabulary else native_char_map['?'] for ch in train_pair[0]],device=device) for train_pair in input]
  #input_encoding.append(EOS_token)
  input_encoding = pad_sequence(input_encoding, batch_first=True,padding_value=0)
  input_encoding=torch.tensor(input_encoding).to(device)
  target_encoding=[torch.tensor([latin_char_map[ch] if ch in latin_character_vocabulary else latin_char_map['?'] for ch in train_pair[1]],device=device)for train_pair in input]
  #target_encoding.append(EOS_token)
  target_encoding = pad_sequence(target_encoding, batch_first=True,padding_value=0)
  target_encoding=torch.tensor(target_encoding).to(device)

  return input_encoding.T,target_encoding.T

input_encoding,target_encoding=prepare_encoding(pairs[0:10],native_enc_map,latin_enc_map,10)


/tmp/ipykernel_31/11156412.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_encoding=torch.tensor(input_encoding).to(device)
/tmp/ipykernel_31/11156412.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_encoding=torch.tensor(target_encoding).to(device)


In [35]:
input_encoding

tensor([[ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 4, 13, 18, 27, 22,  9, 17,  4, 21,  4],
        [ 3, 16, 16,  6,  6,  6, 26,  5, 12,  6],
        [ 6, 20, 22, 23, 25, 23,  4, 23,  3, 22],
        [ 4, 17,  6, 27,  6,  6,  3,  3, 16,  9],
        [25,  3, 20, 12, 20, 20, 13,  6, 15,  6],
        [22, 27, 18,  8, 16, 27,  3, 17,  6,  4],
        [ 6,  6,  6,  6,  6,  6,  6,  8,  7,  6],
        [23,  2, 20,  9,  2, 10, 22,  6,  2, 20],
        [ 6,  0, 25, 26,  0,  3,  6, 20,  0, 23],
        [ 6,  0,  2, 26,  0, 26, 21,  2,  0, 22],
        [22,  0,  0, 25,  0,  2,  6,  0,  0,  6],
        [ 2,  0,  0,  2,  0,  0, 17,  0,  0,  3],
        [ 0,  0,  0,  0,  0,  0,  3,  0,  0,  2],
        [ 0,  0,  0,  0,  0,  0, 27,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0, 26,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  2,  0,  0,  0]], device='cuda:0')

In [36]:
def accuracy(output,target,pair):
  count=0
  
  bat=output.shape[1]
  for i in range(output.shape[1]):
    pred=output[:,i]
    #print(pred)
    tar_pair=pair[i][1]
    tar=target[:,i]
    
    tar=tar.tolist()
    pred=pred.tolist()
    ind=tar.index(2)
    
    tar=tar[:ind]
    pred=pred[:ind]
    
    ans=""
    for j in range(len(tar)):
        ans+=latin_dec_map[pred[j]]
    ans=ans[1:]
    tar_pair=tar_pair[1:]
    tar_pair=tar_pair[:-1]
    #print(ans)
    #print(tar_pair)
    if(ans==tar_pair):
      count+=1
  #print(count)
  return count/output.shape[1]



In [37]:
valid_inp_encoding,valid_target_encoding=prepare_encoding(valid_pairs,native_enc_map,latin_enc_map,len(valid_pairs))
print(accuracy(valid_target_encoding,valid_target_encoding,valid_pairs))

/tmp/ipykernel_31/11156412.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_encoding=torch.tensor(input_encoding).to(device)
/tmp/ipykernel_31/11156412.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_encoding=torch.tensor(target_encoding).to(device)


1.0


In [43]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchtext

class EncoderRNN(nn.Module):
    def __init__(self, n_input,n_embedding, n_hidden, n_layers, cell_type,p,bidirectional):
        super(EncoderRNN, self).__init__()
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.dropout=nn.Dropout(p)
        self.cell_type=cell_type
        self.directions=2 if bidirectional else 1
        self.embedding_layer = nn.Embedding(n_input, n_embedding)
        if cell_type == 'RNN':
            self.rnn_cell = nn.RNN(n_embedding, n_hidden, n_layers,dropout=p,bidirectional=bidirectional)
        elif cell_type == 'LSTM':
            self.rnn_cell = nn.LSTM(n_embedding, n_hidden, n_layers,dropout=p,bidirectional=bidirectional)
        elif cell_type == 'GRU':
            self.rnn_cell = nn.GRU(n_embedding, n_hidden, n_layers,dropout=p,bidirectional=bidirectional)
        #if(bidirectional):


    def forward(self, input, hidden,cell=None):
        #print(input.shape)
        embedded = self.dropout(self.embedding_layer(input))
        #print(embedded.shape)
        output = embedded.unsqueeze(0)
        #hidden.squeeze(0)
        #print(hidden.shape)
        #print(output.shape)
        if(self.cell_type=='LSTM'):
          
            output,(hidden,cell)=self.rnn_cell(output,(hidden,cell))
            return output,hidden,cell
        else:
          output, hidden = self.rnn_cell(output, hidden)
          return output, hidden

    def initHidden(self,batch_size):
        return torch.zeros(self.n_layers*self.directions, batch_size, self.n_hidden, device=device)


class DecoderRNN(nn.Module):
    def __init__(self, n_input,n_embedding, n_hidden, n_output, n_layers, cell_type,p,bidirectional):
        super(DecoderRNN, self).__init__()
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.cell_type=cell_type
        self.dropout=nn.Dropout(p)

        self.embedding_layer = nn.Embedding(n_input, n_embedding)
        if cell_type == 'RNN':
            self.rnn_layer = nn.RNN(n_embedding, n_hidden, n_layers,dropout=p,bidirectional=bidirectional)
        elif cell_type == 'LSTM':
            self.rnn_layer = nn.LSTM(n_embedding, n_hidden,n_layers,dropout=p,bidirectional=bidirectional)
        elif cell_type == 'GRU':
            self.rnn_layer = nn.GRU(n_embedding, n_hidden, n_layers,dropout=p,bidirectional=bidirectional)
        #self.fc = nn.Linear(hidden_size, output_size)
        if(bidirectional):
            self.out = nn.Linear(2*n_hidden, n_output)
        else:
            self.out = nn.Linear(n_hidden, n_output)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden,cell=None):
        output = self.dropout(self.embedding_layer(input))
        output=output.unsqueeze(0)
        output = F.relu(output)
        #print(output.shape)
        #print(hidden.shape)
        #print(cell.shape)
        if(self.cell_type=='LSTM'):
          output,(hidden,cell)=self.rnn_layer(output,(hidden,cell))
          
          output=self.softmax(self.out(output[0]))
          #print(output.shape,"&&&&&&&")
          return output,hidden,cell
        else:
          output, hidden = self.rnn_layer(output, hidden)
          output = self.softmax(self.out(output[0]))
          return output, hidden

    def initHidden(self,batch_size):
        return torch.zeros(self.n_layers,batch_size,self.n_hidden, device=device)


teacher_forcing_ratio = 1

def trainnet(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion,validation=False):
    
    if(not(validation)):
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

    #temp_layer=nn.Linear(2*encoder.n_layers,2*decoder.n_layers,device=device)

    input_length = input_tensor.shape[0]
    target_length = target_tensor.shape[0]
    batch_size=input_tensor.shape[1]
    encoder_hidden = encoder.initHidden(batch_size)
    encoder_cell=encoder.initHidden(batch_size)

    encoder_outputs = torch.zeros(input_length,batch_size,encoder.n_hidden, device=device)
    decoder_outputs=torch.zeros(target_length,batch_size,device=device)

    loss = 0
    if(encoder.cell_type=='LSTM'):
      #encoder_output, encoder_hidden,encoder_cell=encoder(input_tensor[0],encoder_hidden)
      #encoder_outputs[0]=encoder_output[0,0]
      for ei in range(0,input_length):
          encoder_output, encoder_hidden,encoder_cell = encoder(
              input_tensor[ei], encoder_hidden,encoder_cell)
          #print(encoder_cell.shape,"****")
          #encoder_outputs[ei] = encoder_output[0, 0]

    
    else:
      for ei in range(0,input_length):
          encoder_output, encoder_hidden = encoder(
              input_tensor[ei], encoder_hidden)
    
    #decoder_hidden = temp_layer(encoder_hidden.permute(1,2,0))
    #decoder_hidden=decoder_hidden.permute(2,0,1).contiguous()
    #print(encoder_hidden.shape)
    decoder_input = target_tensor[0]
    decoder_hidden=encoder_hidden
    decoder_cell=encoder_cell

    #decoder_cell=temp_layer(encoder_cell.permute(1,2,0))
    #decoder_cell=decoder_cell.permute(2,0,1).contiguous()
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if(validation):
        if encoder.cell_type=='LSTM':
           for di in range(target_length):
                  decoder_output, decoder_hidden,decoder_cell = decoder(
                      decoder_input, decoder_hidden, decoder_cell)
                  topv, topi = decoder_output.topk(1)
                  #print(topi.shape,"UUUUUUUUUUUU")
                  decoder_input = topi.squeeze().detach()
                  decoder_outputs[di]=decoder_input
           return decoder_outputs
        else:
           for di in range(target_length):
                  decoder_output, decoder_hidden= decoder(
                      decoder_input, decoder_hidden)
                  topv, topi = decoder_output.topk(1)
                  #print(topi.shape,"UUUUUUUUUUUU")
                  decoder_input = topi.squeeze().detach()
                  decoder_outputs[di]=decoder_input
           return decoder_outputs 

    else:
      if use_teacher_forcing:
          # Teacher forcing: Feed the target as the next input
          if encoder.cell_type=='LSTM':
            for di in range(target_length):
                #print(decoder_cell.shape)
                decoder_output, decoder_hidden, decoder_cell = decoder(decoder_input, decoder_hidden, decoder_cell)
                loss += criterion(decoder_output, target_tensor[di])
                decoder_input = target_tensor[di] 
          else:
             for di in range(target_length):
                #print(decoder_cell.shape)
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                loss += criterion(decoder_output, target_tensor[di])
                decoder_input = target_tensor[di] 

      else:
          # Without teacher forcing: use its own predictions as the next input
        if encoder.cell_type=='LSTM':
          for di in range(target_length):
              decoder_output, decoder_hidden,decoder_cell = decoder(
                  decoder_input, decoder_hidden, decoder_cell)
              topv, topi = decoder_output.topk(1)
              #print(topi.shape,"UUUUUUUUUUUU")
              decoder_input = topi.squeeze().detach()  # detach from history as input
        else:
            for di in range(target_length):
              decoder_output, decoder_hidden = decoder(
                  decoder_input, decoder_hidden)
              topv, topi = decoder_output.topk(1)
              #print(topi.shape,"UUUUUUUUUUUU")
              decoder_input = topi.squeeze().detach()  # detach from history as input

        loss += criterion(decoder_output, target_tensor[di])
              

      loss.backward()

      encoder_optimizer.step()
      decoder_optimizer.step()

      return loss.item() / target_length

def trainIters(encoder, decoder, epochs, batch_size, learning_rate):
    
   
    print(f"{'epoch': <5}{'training loss' : ^30}{'training accuracy ': <30}{'validation loss ': <30}")
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    
    criterion = nn.NLLLoss()
    for epoch in range(epochs):
      loss=0
      count=0
      sumloss=0
      acc=0
      tot_ans=[]
      for i in range(0,len(pairs),batch_size):
        input_encoding,target_encoding=prepare_encoding(pairs[i:i+batch_size],native_enc_map,latin_enc_map,batch_size)

   
        input_tensor = input_encoding
        target_tensor = target_encoding

        loss = trainnet(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        sumloss+=loss
        count+=1
      

      valid_inp_encoding,valid_target_encoding=prepare_encoding(valid_pairs,native_enc_map,latin_enc_map,len(valid_pairs))
      valid_out=trainnet(valid_inp_encoding, valid_target_encoding, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion,True)
      #valid_out,valid_target=prepare(valid_out,valid_target_encoding)
      print(valid_out.shape,valid_target_encoding.shape)
      valid_acc=accuracy(valid_out,valid_target_encoding,valid_pairs)
      #print(valid_acc)
      wandb.log({"validation_accuracy": valid_acc,"training_loss": sumloss/count,"epoch":epoch})
      print(f"{epoch : <5}{sumloss/count : ^30}{acc: <30}{valid_acc : <30}")
     

''''hidden_size = 256
num_layers=2
enc_dropout=0
dec_dropout=0
encoder_embedding_size=256
decoder_embedding_size=256
output_size=len(latin_character_vocabulary)
input_size_encoder=len(native_character_vocabulary)
input_size_decoder=len(latin_character_vocabulary)

encoder1=EncoderRNN(input_size_encoder,encoder_embedding_size,hidden_size,2,'LSTM',enc_dropout,True).to(device)
decoder1=DecoderRNN(input_size_decoder, decoder_embedding_size, hidden_size, output_size, 2, 'LSTM', dec_dropout,True).to(device)'''''


#trainIters(encoder1, decoder1, 7,16,0.001)

"'hidden_size = 256\nnum_layers=2\nenc_dropout=0\ndec_dropout=0\nencoder_embedding_size=256\ndecoder_embedding_size=256\noutput_size=len(latin_character_vocabulary)\ninput_size_encoder=len(native_character_vocabulary)\ninput_size_decoder=len(latin_character_vocabulary)\n\nencoder1=EncoderRNN(input_size_encoder,encoder_embedding_size,hidden_size,2,'LSTM',enc_dropout,True).to(device)\ndecoder1=DecoderRNN(input_size_decoder, decoder_embedding_size, hidden_size, output_size, 2, 'LSTM', dec_dropout,True).to(device)"

In [44]:
def network_train(epochs,n_layers,n_embedding,lr,batch_size,cell_type,bidirectional,enc_dropout,dec_dropout,n_hidden):
    input_size_encoder=len(native_character_vocabulary)
    input_size_decoder=len(latin_character_vocabulary)
    
    output_size=len(latin_character_vocabulary)
    encoder_net=EncoderRNN(input_size_encoder,n_embedding,n_hidden,n_layers,cell_type,enc_dropout,bidirectional).to(device)
    decoder_net=DecoderRNN(input_size_decoder, n_embedding, n_hidden, output_size, n_layers, cell_type, dec_dropout,bidirectional).to(device)


    trainIters(encoder_net, decoder_net, epochs,batch_size,lr)
    
    

In [42]:
sweep_config = {
    'method': 'bayes',     
    'metric': { 'name': 'validation_accuracy', 'goal': 'maximize'},
    'parameters': {'epoch': {'values': [5, 10,15,20] } ,
                   'n_embedding': { 'values': [16,32, 64, 256] },
                  'n_layers':{ 'values':[1,2,3]  },
                   'lr': {  'values': [1e-2,1e-3,1e-4]},
                  'batch_size': {'values': [16, 32, 64] },
                  'cell_type': {'values': ['RNN','LSTM','GRU'] },
                    'bidirectional': {'values': [True,False] },
                    'enc_dropout': {'values': [0,0.1,0.2,0.3] },
                   'dec_dropout': {'values': [0,0.1,0.2,0.3] },
                   'n_hidden': {'values': [16,32,64,256] },
                 
                   },
    
  }

def train():
    run=wandb.init(project="assignment3",entity="cs22m025",reinit=True)
    config = wandb.config
    #took reference from wandb document
    wandb.run.name="ep_{}_hl_{}_em_{}_lr_{}_bt_{}_cl_{}_bi_{}_edrp_{}_ddrp_{}_hl_{}".format(config.epoch,config.n_layers,config.n_embedding,config.lr,config.batch_size,config.cell_type,config.bidirectional, config.enc_dropout,config.dec_dropout,config.n_hidden)
    wandb.run.save
    tot_ans=network_train(config.epoch,config.n_layers,config.n_embedding,config.lr,config.batch_size,config.cell_type,config.bidirectional, config.enc_dropout,config.dec_dropout,config.n_hidden)
    run.finish()


sweep_id = wandb.sweep(sweep=sweep_config, project="assignment3")

wandb.agent(sweep_id, function=train,count=30)

Create sweep with ID: zv52kwhb
Sweep URL: https://wandb.ai/cs22m025/assignment3/sweeps/zv52kwhb


wandb: Agent Starting Run: spaz303e with config:
wandb: 	batch_size: 16
wandb: 	bidirectional: True
wandb: 	cell_type: RNN
wandb: 	dec_dropout: 0.3
wandb: 	enc_dropout: 0
wandb: 	epoch: 15
wandb: 	lr: 0.01
wandb: 	n_embedding: 64
wandb: 	n_hidden: 16
wandb: 	n_layers: 3
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


epoch        training loss         training accuracy             validation loss               


/tmp/ipykernel_31/11156412.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_encoding=torch.tensor(input_encoding).to(device)
/tmp/ipykernel_31/11156412.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_encoding=torch.tensor(target_encoding).to(device)


Run spaz303e errored: TypeError('train() takes 0 positional arguments but 8 were given')
wandb: ERROR Run spaz303e errored: TypeError('train() takes 0 positional arguments but 8 were given')
